# 🔹UFC Fight Predictor ETL

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 1. Import Libraries and Setup Environment

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Get the current working directory
current_dir = os.getcwd()

# Navigate to the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Import from /src
sys.path.append(os.path.join(project_root, 'src'))
from helpers import *

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 2. Load Data

In [2]:
# Define the path to the CSV file
file_path = os.path.join(project_root, 'data', 'raw', 'ufc_raw.csv')

# Load the CSV into a DataFrame
try:
    ufc_raw = pd.read_csv(file_path)
    print_header(f"Data successfully loaded: {ufc_raw.shape[0]} rows, {ufc_raw.shape[1]} columns.", color='bright_green')
except Exception as e:
    print_header(f"Error loading training data: {e}", color='bright_red')

╔═════════════════════════════════════════════════════╗
║  Data successfully loaded: 6541 rows, 118 columns.  ║
╚═════════════════════════════════════════════════════╝


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 3. Preview

In [3]:
# Preview the first few records
display(ufc_raw.head())

# General dataset information
ufc_raw.info()

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,FinishDetails,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Colby Covington,Joaquin Buckley,205.0,-250.0,205.0000,40.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,NaN,3.0,4:42,882.0,300.0,175.0,1800.0,2000.0,1100.0,150.0
1,Cub Swanson,Billy Quarantillo,124.0,-148.0,124.0000,67.5676,2024-12-14,"Tampa, Florida, USA",USA,Red,...,Punch,3.0,1:36,696.0,250.0,NaN,1800.0,NaN,450.0,NaN
2,Manel Kape,Bruno Silva,-395.0,310.0,25.3165,310.0000,2024-12-14,"Tampa, Florida, USA",USA,Red,...,Punches,3.0,1:57,717.0,-105.0,550.0,900.0,1800.0,225.0,1100.0
3,Vitor Petrino,Dustin Jacoby,-340.0,270.0,29.4118,270.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,Punch,3.0,3:44,824.0,240.0,500.0,550.0,3000.0,110.0,800.0
4,Adrian Yanez,Daniel Marcos,185.0,-225.0,185.0000,44.4444,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,NaN,3.0,5:00,900.0,450.0,150.0,2200.0,2200.0,450.0,200.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6541 entries, 0 to 6540
Columns: 118 entries, RedFighter to BKOOdds
dtypes: bool(1), float64(60), int64(43), object(14)
memory usage: 5.8+ MB


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 4. Check 

In [4]:
# Null values check
nulls = ufc_raw.isnull().sum()
print("\nNull values per column:\n", nulls[nulls > 0])

# Duplicate analysis
duplicates = ufc_raw.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")


Null values per column:
 RedOdds                   227
BlueOdds                  226
RedExpectedValue          227
BlueExpectedValue         226
BlueAvgSigStrLanded       930
BlueAvgSigStrPct          765
BlueAvgSubAtt             832
BlueAvgTDLanded           833
BlueAvgTDPct              842
BlueStance                  3
RedAvgSigStrLanded        455
RedAvgSigStrPct           357
RedAvgSubAtt              357
RedAvgTDLanded            357
RedAvgTDPct               367
EmptyArena               1499
BMatchWCRank             5339
RMatchWCRank             4760
RWFlyweightRank          6445
RWFeatherweightRank      6532
RWStrawweightRank        6395
RWBantamweightRank       6387
RHeavyweightRank         6355
RLightHeavyweightRank    6357
RMiddleweightRank        6359
RWelterweightRank        6349
RLightweightRank         6357
RFeatherweightRank       6364
RBantamweightRank        6360
RFlyweightRank           6352
RPFPRank                 6288
BWFlyweightRank          6468
BWFeatherweigh

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 5. Data Cleaning

### Null Values

In [5]:
# Drop columns with too many null values (threshold: 1000)
ufc_deploy = ufc_raw.copy()
threshold = 1000
cols_to_drop = [col for col in ufc_deploy.columns if ufc_deploy[col].isnull().sum() > threshold]
for col in cols_to_drop:
    print('Dropping:', col)
ufc_deploy.drop(columns=cols_to_drop, inplace=True)

# Drop rows with any remaining missing values
print(f"➡️ Before dropna: {ufc_deploy.shape}")
ufc_deploy.dropna(inplace=True)
print(f"✅ After dropna: {ufc_deploy.shape}")

Dropping: EmptyArena
Dropping: BMatchWCRank
Dropping: RMatchWCRank
Dropping: RWFlyweightRank
Dropping: RWFeatherweightRank
Dropping: RWStrawweightRank
Dropping: RWBantamweightRank
Dropping: RHeavyweightRank
Dropping: RLightHeavyweightRank
Dropping: RMiddleweightRank
Dropping: RWelterweightRank
Dropping: RLightweightRank
Dropping: RFeatherweightRank
Dropping: RBantamweightRank
Dropping: RFlyweightRank
Dropping: RPFPRank
Dropping: BWFlyweightRank
Dropping: BWFeatherweightRank
Dropping: BWStrawweightRank
Dropping: BWBantamweightRank
Dropping: BHeavyweightRank
Dropping: BLightHeavyweightRank
Dropping: BMiddleweightRank
Dropping: BWelterweightRank
Dropping: BLightweightRank
Dropping: BFeatherweightRank
Dropping: BBantamweightRank
Dropping: BFlyweightRank
Dropping: BPFPRank
Dropping: FinishDetails
Dropping: RedDecOdds
Dropping: BlueDecOdds
Dropping: RSubOdds
Dropping: BSubOdds
Dropping: RKOOdds
Dropping: BKOOdds
➡️ Before dropna: (6541, 82)
✅ After dropna: (4719, 82)


In [6]:
# Drop columns with too many null values (threshold: 300)
threshold = 300
cols_to_drop = [col for col in ufc_raw.columns if ufc_raw[col].isnull().sum() > threshold]
for col in cols_to_drop:
    print('Dropping:', col)
ufc_raw.drop(columns=cols_to_drop, inplace=True)

# Drop rows with any remaining missing values
print(f"➡️ Before dropna: {ufc_raw.shape}")
ufc_raw.dropna(inplace=True)
print(f"✅ After dropna: {ufc_raw.shape}")

Dropping: BlueAvgSigStrLanded
Dropping: BlueAvgSigStrPct
Dropping: BlueAvgSubAtt
Dropping: BlueAvgTDLanded
Dropping: BlueAvgTDPct
Dropping: RedAvgSigStrLanded
Dropping: RedAvgSigStrPct
Dropping: RedAvgSubAtt
Dropping: RedAvgTDLanded
Dropping: RedAvgTDPct
Dropping: EmptyArena
Dropping: BMatchWCRank
Dropping: RMatchWCRank
Dropping: RWFlyweightRank
Dropping: RWFeatherweightRank
Dropping: RWStrawweightRank
Dropping: RWBantamweightRank
Dropping: RHeavyweightRank
Dropping: RLightHeavyweightRank
Dropping: RMiddleweightRank
Dropping: RWelterweightRank
Dropping: RLightweightRank
Dropping: RFeatherweightRank
Dropping: RBantamweightRank
Dropping: RFlyweightRank
Dropping: RPFPRank
Dropping: BWFlyweightRank
Dropping: BWFeatherweightRank
Dropping: BWStrawweightRank
Dropping: BWBantamweightRank
Dropping: BHeavyweightRank
Dropping: BLightHeavyweightRank
Dropping: BMiddleweightRank
Dropping: BWelterweightRank
Dropping: BLightweightRank
Dropping: BFeatherweightRank
Dropping: BBantamweightRank
Dropping: 

In [7]:
# Null values check
nulls = ufc_raw.isnull().sum()
print("\nNull values per column:\n", nulls[nulls > 0])


Null values per column:
 Series([], dtype: int64)


In [8]:
ufc_raw.columns

Index(['RedFighter', 'BlueFighter', 'RedOdds', 'BlueOdds', 'RedExpectedValue',
       'BlueExpectedValue', 'Date', 'Location', 'Country', 'Winner',
       'TitleBout', 'WeightClass', 'Gender', 'NumberOfRounds',
       'BlueCurrentLoseStreak', 'BlueCurrentWinStreak', 'BlueDraws',
       'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought',
       'BlueTotalTitleBouts', 'BlueWinsByDecisionMajority',
       'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous',
       'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage',
       'BlueWins', 'BlueStance', 'BlueHeightCms', 'BlueReachCms',
       'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak',
       'RedDraws', 'RedLongestWinStreak', 'RedLosses', 'RedTotalRoundsFought',
       'RedTotalTitleBouts', 'RedWinsByDecisionMajority',
       'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous', 'RedWinsByKO',
       'RedWinsBySubmission', 'RedWinsByTKODoctorStoppage', 'RedWins',
       'RedStance', 'Red

### Incongruent Data

In [9]:
print(ufc_raw[['RedReachCms', 'BlueReachCms']].describe())

       RedReachCms  BlueReachCms
count  6062.000000   6062.000000
mean    182.542097    182.342446
std      11.060124     11.139982
min     147.320000      0.000000
25%     175.260000    175.260000
50%     182.880000    182.880000
75%     190.500000    190.500000
max     214.630000    213.360000


In [10]:
# Reemplazar ceros o nulos en los alcances antes de dividir
ufc_raw['RedReachCms'] = ufc_raw['RedReachCms'].replace(0, ufc_raw['RedReachCms'].mean())
ufc_raw['BlueReachCms'] = ufc_raw['BlueReachCms'].replace(0, ufc_raw['BlueReachCms'].mean())

In [11]:
# Open Stance is incorrect
ufc_raw[ufc_raw['RedStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish
6051,Krzysztof Soszynski,Igor Pokrajac,-255.0,235.0,39.2157,235.0000,2011-12-10,"Toronto, Ontario, Canada",Canada,Blue,...,-1,-1,-2.54,-2.54,2,-15.5000,-0.5500,0.6250,neither,KO/TKO
6366,Krzysztof Soszynski,Goran Reljic,-108.0,-102.0,92.5926,98.0392,2010-11-13,"Oberhausen, North Rhine-Westphalia, Germany",Germany,Red,...,-1,-2,5.08,10.16,7,-4.5000,0.8333,0.8333,neither,U-DEC
6448,Krzysztof Soszynski,Stephan Bonnar,-190.0,175.0,52.6316,175.0000,2010-07-03,"Las Vegas, Nevada, USA",USA,Blue,...,-1,0,7.62,2.54,-1,3.2000,0.4000,0.1273,neither,KO/TKO
6511,Nate Quarry,Jorge Rivera,-230.0,190.0,43.4783,190.0000,2010-03-31,"Charlotte, North Carolina, USA",USA,Blue,...,-2,0,2.54,2.54,0,-2.8182,0.0909,0.4343,neither,KO/TKO


In [12]:
ufc_raw = ufc_raw[ufc_raw['RedStance'] != 'Open Stance']

In [13]:
# Open Stance is incorrect
ufc_raw[ufc_raw['RedStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish


In [14]:
# Open Stance is incorrect
ufc_raw[ufc_raw['BlueStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish
6216,Mike Massenzio,Krzysztof Soszynski,265.0,-325.0,265.0,30.7692,2011-06-11,"Vancouver, British Columbia, Canada",Canada,Blue,...,2,1,-2.54,10.16,-5,28.4762,-0.8095,-1.5714,neither,U-DEC


In [15]:
ufc_raw = ufc_raw[ufc_raw['BlueStance'] != 'Open Stance']

In [16]:
# Open Stance is incorrect
ufc_raw[ufc_raw['BlueStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish


In [17]:
# Open Stance is incorrect
ufc_raw[ufc_raw['RedStance'] == 'Open Stance']

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,KODif,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish


## 6. Create Fight Stance Columns
- If both fighters have the same fighting stance, the bout is considered a Closed Stance matchup. If their stances differ, it is classified as an Open Stance matchup.

In [18]:
# Create column FightStance according Stances matches.
ufc_raw['FightStance'] = np.where(
    ufc_raw['BlueStance'] == ufc_raw['RedStance'],
    'Closed Stance',
    'Open Stance'
)

In [19]:
ufc_preview = ufc_raw[ufc_raw['FightStance'] == 'Open Stance']

In [20]:
ufc_preview2 = ufc_raw[ufc_raw['FightStance'] == 'Closed Stance']

In [21]:
ufc_preview[['FightStance', 'BlueStance', 'RedStance']]

,FightStance,BlueStance,RedStance
0,Open Stance,Southpaw,Orthodox
2,Open Stance,Orthodox,Southpaw
6,Open Stance,Switch,Southpaw
10,Open Stance,Orthodox,Southpaw
11,Open Stance,Southpaw,Orthodox
...,...,...,...
6524,Open Stance,Orthodox,Southpaw
6525,Open Stance,Orthodox,Southpaw
6528,Open Stance,Orthodox,Switch
6529,Open Stance,Orthodox,Southpaw


In [22]:
ufc_preview2[['FightStance', 'BlueStance', 'RedStance']]

,FightStance,BlueStance,RedStance
1,Closed Stance,Orthodox,Orthodox
3,Closed Stance,Orthodox,Orthodox
4,Closed Stance,Orthodox,Orthodox
5,Closed Stance,Orthodox,Orthodox
7,Closed Stance,Orthodox,Orthodox
...,...,...,...
6535,Closed Stance,Orthodox,Orthodox
6536,Closed Stance,Orthodox,Orthodox
6538,Closed Stance,Orthodox,Orthodox
6539,Closed Stance,Orthodox,Orthodox


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## 7. Check Clean Data

In [23]:
# Null values check
nulls = ufc_raw.isnull().sum()
print("\nNull values per column:\n", nulls[nulls > 0])

# Duplicate analysis
duplicates = ufc_raw.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")


Null values per column:
 Series([], dtype: int64)

Duplicate rows: 0


In [24]:
# Preview the first few records
display(ufc_raw.head())
display(ufc_raw.columns)
# Para ver los tipos de dato de cada columna:
display(ufc_raw.dtypes)

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,SubDif,HeightDif,ReachDif,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish,FightStance
0,Colby Covington,Joaquin Buckley,205.0,-250.0,205.0000,40.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,-2,-2.54,10.16,-6,0.25,-0.2,-1.83,Red,KO/TKO,Open Stance
1,Cub Swanson,Billy Quarantillo,124.0,-148.0,124.0000,67.5676,2024-12-14,"Tampa, Florida, USA",USA,Red,...,-1,5.08,0.00,-5,2.69,0.7,0.20,neither,KO/TKO,Closed Stance
2,Manel Kape,Bruno Silva,-395.0,310.0,25.3165,310.0000,2024-12-14,"Tampa, Florida, USA",USA,Red,...,1,-2.54,-7.62,3,-1.12,-0.2,1.72,Red,KO/TKO,Open Stance
3,Vitor Petrino,Dustin Jacoby,-340.0,270.0,29.4118,270.0000,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,-1,2.54,-2.54,9,2.68,-0.8,-3.62,neither,KO/TKO,Closed Stance
4,Adrian Yanez,Daniel Marcos,185.0,-225.0,185.0000,44.4444,2024-12-14,"Tampa, Florida, USA",USA,Blue,...,0,0.00,-2.54,0,-0.57,0.0,0.25,neither,S-DEC,Closed Stance


Index(['RedFighter', 'BlueFighter', 'RedOdds', 'BlueOdds', 'RedExpectedValue',
       'BlueExpectedValue', 'Date', 'Location', 'Country', 'Winner',
       'TitleBout', 'WeightClass', 'Gender', 'NumberOfRounds',
       'BlueCurrentLoseStreak', 'BlueCurrentWinStreak', 'BlueDraws',
       'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought',
       'BlueTotalTitleBouts', 'BlueWinsByDecisionMajority',
       'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous',
       'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage',
       'BlueWins', 'BlueStance', 'BlueHeightCms', 'BlueReachCms',
       'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak',
       'RedDraws', 'RedLongestWinStreak', 'RedLosses', 'RedTotalRoundsFought',
       'RedTotalTitleBouts', 'RedWinsByDecisionMajority',
       'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous', 'RedWinsByKO',
       'RedWinsBySubmission', 'RedWinsByTKODoctorStoppage', 'RedWins',
       'RedStance', 'Red

RedFighter           object
BlueFighter          object
RedOdds             float64
BlueOdds            float64
RedExpectedValue    float64
                     ...   
AvgSubAttDif        float64
AvgTDDif            float64
BetterRank           object
Finish               object
FightStance          object
Length: 70, dtype: object

## 8. Create the target value: **0** (Fighter Red wins) or **1** (Fighter Blue wins)

In [25]:
ufc_raw['label'] = ufc_raw['Winner'].apply(lambda x: 1 if x == 'Blue' else 0)
ufc_raw=ufc_raw.drop('Winner', axis=1)

ufc_deploy['label'] = ufc_deploy['Winner'].apply(lambda x: 1 if x == 'Blue' else 0)
ufc_deploy=ufc_deploy.drop('Winner', axis=1)

## 9. Save

In [26]:
# Save the cleaned file
ufc_raw.to_csv(f'{project_root}/data/processed/ufc_etl.csv', index=False)
print_header("ETL file saved as 'ufc_etl.csv'.", color = 'bright_green')

╔════════════════════════════════════╗
║  ETL file saved as 'ufc_etl.csv'.  ║
╚════════════════════════════════════╝


In [27]:
# Save the cleaned file
ufc_deploy.to_csv(f'{project_root}/data/processed/ufc_deploy.csv', index=False)
print_header("Deploy file saved as 'ufc_deploy.csv'.", color = 'bright_green')

╔══════════════════════════════════════════╗
║  Deploy file saved as 'ufc_deploy.csv'.  ║
╚══════════════════════════════════════════╝


<div style="text-align: center;">
     <img src="../img/ufc_logo.png" width="800" /> 
</div>